# ML.Net - AnomalyDetection

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

## Using C# Class

In [2]:
using System;
using Microsoft.ML;
using Microsoft.ML.Data;
using System.Linq;

## Declare data-classes for input data and predictions

In [3]:
public class EnergyData
{
    [LoadColumn(0)]
    public DateTime Date { get; set; }

    [LoadColumn(1)]
    public float Energy { get; set; }
}

public class EnergyPrediction
{
    [VectorType(2)]
    public double[] Prediction { get; set; }
}

public class AnomalyResult
{
    public bool PredictedLabel { get; set; }
    public float Score { get; set; }
}

## Evaluate

In [4]:
var context = new MLContext();

var data = context.Data.LoadFromTextFile<EnergyData>("./datasets/energy/energy_hourly.csv", 
    hasHeader: true, 
    separatorChar: ',');

var pipeline = context.Transforms.DetectSpikeBySsa(nameof(EnergyPrediction.Prediction), nameof(EnergyData.Energy),
    confidence: 98, trainingWindowSize: 90, seasonalityWindowSize: 30, pvalueHistoryLength: 30);

var transformedData = pipeline.Fit(data).Transform(data);

var predictions = context.Data.CreateEnumerable<EnergyPrediction>(transformedData, reuseRowObject: false).ToList();

var energy = data.GetColumn<float>("Energy").ToArray();
var date = data.GetColumn<DateTime>("Date").ToArray();

Console.WriteLine("Anomalies:");
for (int i = 0; i < predictions.Count(); i++)
{
    if (predictions[i].Prediction[0] == 1)
    {
        Console.WriteLine("{0}\t{1:0.0000}\t{2:0.00}\t{3:0.00}\t{4:0.00}",
            date[i], energy[i], predictions[i].Prediction[0], predictions[i].Prediction[1], predictions[i].Prediction[2]);
    }
}

Console.ReadLine();

Unhandled exception: (7,35): error CS1061: ‘TransformsCatalog’ não contém uma definição para "DetectSpikeBySsa" e não foi possível encontrar nenhum método de extensão "DetectSpikeBySsa" que aceite um primeiro argumento do tipo ‘TransformsCatalog’ (você está se esquecendo de usar uma diretiva ou uma referência de assembly?)